In [5]:
from datetime import datetime
import time

import numpy as np
import mlagents
from mlagents_envs.environment import UnityEnvironment

from mlagents.torch_utils import torch, default_device, set_torch_config
from mlagents.plugins.bvh_utils.visualize import skeletons_plot
import mlagents.plugins.bvh_utils.lafan_utils as utils
from mlagents.plugins.skeleton_aware_op.dataset import TemporalMotionData, SkeletonInfo
from mlagents.plugins.skeleton_aware_op.options import get_options as get_options_sk

from mlagents_envs.side_channel.side_channel import (
    SideChannel,
    IncomingMessage,
    OutgoingMessage,
)
import uuid
import matplotlib.pyplot as plt
from mlagents.plugins.bvh_utils.Quaternions import Quaternions
%matplotlib qt

torch.set_printoptions(precision=4)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [2]:
def reorder(xyz, data):

    if isinstance(data, np.ndarray):
        temp = data.copy()
    elif isinstance(data, torch.Tensor):
        temp = data.clone()

    # reorder to fit the frame of reference
    for ind, letter in enumerate(xyz):
        
        if letter == 'x':
            data[..., ind] = temp[..., 0]
        elif letter == 'y':
            data[..., ind] = temp[..., 1]
        elif letter == 'z':
            data[..., ind] = temp[..., 2]

    return data

class Skeleton_SideChannel(SideChannel):
    def __init__(self) -> None:
        super().__init__(uuid.UUID("621f0a70-4f87-11ea-a6bf-784f4387d1f7"))

    def on_message_received(self, msg: IncomingMessage) -> None:
        """
        Note: We must implement this method of the SideChannel interface to
        receive messages from Unity
        """
        self.msg = msg.read_float32_list()
        # We simply read a string from the message and print it.
        print(msg.read_float32_list())

    def send_string(self, data: str) -> None:
        pass


In [3]:
# Setup Unity Environment + sidechannel
skeleton_sidechannel = Skeleton_SideChannel()

try:
    env.close()
except:
    pass

# filename = None enables to communicate directly with the unity editor
env = UnityEnvironment(file_name=None, seed=1, side_channels=[skeleton_sidechannel])
env.reset()

# Setup LaFan skeleton data
# Get skeleton data from dataset
motion_path = "../ml-agents/mlagents/plugins/skeleton_aware_op/data/LaFan/Real_Data/"
# motion_path = "../../LaFan_BVH/rotation_test/"
skdata = SkeletonInfo(motion_path, xyz='zxy')

# get lafan motion rotations
motion_path = "../ml-agents/mlagents/plugins/skeleton_aware_op/data/LaFan/Real_Data/"
lafan_dataset = TemporalMotionData(motion_path, recalculate_mean_var=True, normalize_data=True, xyz='zxy', rot_order=None)
skdata = lafan_dataset.skdata

options_sk = get_options_sk()


[]
[DATASET] final data shape :  torch.Size([29, 88, 16])
[DATASET] input concatenated shape:  torch.Size([464, 88])
[DATASET] mean shape:  torch.Size([1, 88])
[DATASET] var shape:  torch.Size([1, 88])
[DATASET] mean var shape : torch.Size([2, 88, 1])


In [4]:
behavior_name = list(env.behavior_specs)[0]
spec = env.behavior_specs[behavior_name]
env.reset()

# get action and obs dimension
state_dim = spec.observation_specs[0].shape[0]
action_dim = spec.action_spec.continuous_size

decision_steps, terminal_steps = env.get_steps(behavior_name)

In [5]:
# Generate an action for all agents
action = spec.action_spec.random_action(len(decision_steps))

# Set the actions
env.set_actions(behavior_name, action)

# Move the simulation forward
env.step()


In [6]:
features_per_joint = 13
obs = decision_steps.obs[0]

# extract only the observation corresponding to the joints
joint_features = obs[0,3:]
joint_features = joint_features.reshape(-1, 13)

init_rotations = torch.tensor(skeleton_sidechannel.msg[:22*4]).float().reshape(22,4)
init_positions = torch.tensor(skeleton_sidechannel.msg[22*4:]).float().reshape(22,3)

# extract information from the observation vector
velocity = torch.tensor(joint_features[:,:3].copy()).float()
angular_vel = torch.tensor(joint_features[:,3:6].copy()).float()
positions = torch.tensor(joint_features[:,6:9].copy()).float()
rotations = torch.tensor(joint_features[:,9:].copy()).float()

# unity rotation is x,y,z,w and lafan is w,x,y,z, change to the latter
temp = rotations[:,3].clone()
rotations[:,1:] = rotations[:,:3] 
rotations[:,0] = temp


In [11]:
scale = 100

# unit rotations, used to get ini
unit_rotations = torch.tensor([[1.,0.,0.,0.]])
unit_rotations = unit_rotations.repeat(22,1)

# get the unity offset from the init positions -> local position
unity_offsets = init_positions.clone()
unity_offsets[0,:] = torch.tensor([0,0,0])

# get lafan offset, scale corrected
lafan_offsets = skdata.offsets.clone()/scale

# get the current unity local pose (unit global rotation)
rotations_local = rotations.clone()
rotations_local[0,:] = torch.tensor([1.,0.,0.,0.]).float()
_, unity_pos_local = utils.quat_fk(rotations_local, unity_offsets, skdata.parents)

# get some data from the dataset
motion_data = lafan_dataset[20:22]
motion_data = lafan_dataset.denormalize(motion_data)

# we want to add a rotation offset on top of the local pos to align it with unity
# for dataset zxy -> euler rotation ->(0,0,-90) 
rotation_offset = torch.tensor([  0.7071068 ,0, 0, 0.7071068])#Quaternions.from_euler(np.array([0,0,0]), 'xyz').qs)
lafan_pos, lafan_pos_local, lafan_glob, lafan_glob_velo, lafan_vel = utils.get_pos_info_from_raw(motion_data, skdata, lafan_offsets, options_sk, norm_rot=False, rotation_offset=rotation_offset)

limits = [[-1,1],[-1,1],[-1,1]]
fig, ax = skeletons_plot([unity_pos_local.cpu().detach(), lafan_pos_local[0,0].cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
plt.show()


torch.Size([2, 16, 4])
torch.Size([2, 16, 4])


In [8]:
scale = 100

# unit rotations, used to get ini
unit_rotations = torch.tensor([[1.,0.,0.,0.]])
unit_rotations = unit_rotations.repeat(22,1)

# get the unity offset from the init positions -> local position
unity_offsets = init_positions.clone()
unity_offsets[0,:] = torch.tensor([0,0,0])

# get lafan offset that aligns
lafan_offsets = skdata.offsets.clone()/scale

# initial offsets, obtained by doing fk with unit rotations
_, unity_init_offset = utils.quat_fk(unit_rotations, unity_offsets, skdata.parents)
_, lafan_init_offset = utils.quat_fk(unit_rotations, lafan_offsets, skdata.parents)

# t pose using the initial rotation for both dataset
_, unity_t_pose = utils.quat_fk(init_rotations, unity_offsets, skdata.parents)
_, lafan_t_pose = utils.quat_fk(init_rotations, lafan_offsets, skdata.parents)


limits = [[-1,1],[-1,1],[-1,1]]
fig, ax = skeletons_plot([unity_init_offset.cpu().detach(), lafan_init_offset.cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
plt.show()
limits = [[-1,1],[-1,1],[-1,1]]
fig, ax = skeletons_plot([unity_t_pose.cpu().detach(), lafan_t_pose.cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
plt.show()

In [9]:
# t pose using the initial rotation for both dataset
_, unity_pos = utils.quat_fk(rotations, unity_offsets, skdata.parents)
_, lafan_pos = utils.quat_fk(rotations, lafan_offsets, skdata.parents)

rotations_local = rotations.clone()
rotations_local[0,:] = torch.tensor([1.,0.,0.,0.]).float()
_, unity_pos_local = utils.quat_fk(rotations_local, unity_offsets, skdata.parents)

limits = [[-1,1],[-1,1],[-1,1]]
fig, ax = skeletons_plot([unity_pos.cpu().detach(), lafan_pos.cpu().detach()], [skdata.edges, skdata.edges], ['g', 'b'], limits=limits, return_plot=True)
plt.show()

In [6]:
# Setup LaFan skeleton data
# Get skeleton data from dataset
motion_path = "../ml-agents/mlagents/plugins/skeleton_aware_op/data/LaFan/Real_Data/"
# motion_path = "../../LaFan_BVH/rotation_test/"
skdata = SkeletonInfo(motion_path, xyz='zxy')

# get lafan motion rotations
motion_path = "../ml-agents/mlagents/plugins/skeleton_aware_op/data/LaFan/Real_Data/"
lafan_dataset = TemporalMotionData(motion_path, recalculate_mean_var=True, normalize_data=True, xyz='zxy', rot_order=None)
skdata = lafan_dataset.skdata

options_sk = get_options_sk()


[DATASET] final data shape :  torch.Size([979, 88, 16])
[DATASET] input concatenated shape:  torch.Size([15664, 88])
[DATASET] mean shape:  torch.Size([1, 88])
[DATASET] var shape:  torch.Size([1, 88])
[DATASET] mean var shape : torch.Size([2, 88, 1])


In [13]:
motion_data = lafan_dataset[20:22]
print(motion_data.shape)
motion_data = lafan_dataset.denormalize(motion_data)

# get lafan offset that aligns
lafan_offsets = skdata.offsets.clone()/100
# lafan_offsets_r = lafan_offsets.reshape(1, 1, skdata.offsets.shape[0], skdata.offsets.shape[1])
# lafan_offsets_r = lafan_offsets_r.repeat(2, 16, 1, 1)

# we want to add a rotation offset on top of the local pos to align it with unity
# for dataset zxy -> euler rotation ->(0,0,-90) 
# rotation_offset = torch.tensor([  0.7071068 ,0, 0, 0.7071068])#Quaternions.from_euler(np.array([0,0,0]), 'xyz').qs)
# rotation_offset = torch.tensor([  1. ,0., 0., 0.])#Quaternions.from_euler(np.array([0,0,0]), 'xyz').qs)
# rotation_offset = torch.tensor([  0.7071068 ,0.7071068, 0., 0.])#Quaternions.from_euler(np.array([0,0,0]), 'xyz').qs)
rotation_offset = torch.tensor([ 0.5 ,-0.5, 0.5, 0.5])#Quaternions.from_euler(np.array([0,0,0]), 'xyz').qs)

lafan_pos, lafan_pos_local, lafan_glob, lafan_glob_velo, lafan_vel = utils.get_pos_info_from_raw(motion_data, skdata, lafan_offsets, options_sk, norm_rot=False, rotation_offset=rotation_offset)


limits = [[-1,1],[-1,1],[-1,1]]
fig, ax = skeletons_plot([ lafan_pos_local[0,0].cpu().detach()], [skdata.edges], ['b'], limits=limits, return_plot=True)
plt.show()

torch.Size([2, 88, 16])
torch.Size([2, 16, 4])
torch.Size([2, 16, 4])


In [12]:
env.close()